In [1]:
!pip install pandas numpy scikit-learn tensorflow openpyxl

  Using cached pandas-2.3.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.4.0-cp313-cp313-win_amd64.whl.metadata (6.6 kB)
  Using cached scikit_learn-1.8.0-cp313-cp313-win_amd64.whl.metadata (11 kB)
  Using cached tensorflow-2.20.0-cp313-cp313-win_amd64.whl.metadata (4.6 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached scipy-1.16.3-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.12.19-py2.py3-none-any.whl.metadata (1.0 kB)
  Using cached gast-0.7.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached google_pasta-

In [2]:
import os
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)


d:\SLIIT Uni\Y4 S1\Research\Research Project\New train Model\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [3]:
base_dir = r"D:\SLIIT Uni\Y4 S1\Research\Research Project\New train Model\backend\Datasets"
paths = [os.path.join(base_dir, f"Dataset{i}.xlsx") for i in range(1, 6)]

dfs = []
for p in paths:
    df = pd.read_excel(p)
    if "Estimated_Clicks" in df.columns:
        df = df.rename(columns={"Estimated_Clicks": "clicks"})
    dfs.append(df)

full = pd.concat(dfs, ignore_index=True)
print("Dataset shape:", full.shape)


Dataset shape: (125000, 28)


In [4]:
targets = ["likes", "comments", "shares", "clicks", "timing_quality_score"]
full = full.dropna(subset=targets)

full["caption"] = full["caption"].fillna("").astype(str)
full["content"] = full["content"].fillna("").astype(str)
full["text"] = full["caption"] + " " + full["content"]

# PLATFORM (3 categories)
full["platform"] = full["platform"].astype(str).str.strip()
platform_cat = full["platform"].astype("category")
full["platform_id"] = platform_cat.cat.codes
platform_map = dict(enumerate(platform_cat.cat.categories))
inv_platform_map = {v: k for k, v in platform_map.items()}
print("Platform map:", platform_map)

# TIME FEATURES
full["post_hour"] = pd.to_datetime(full["post_time"], errors="coerce").dt.hour
full["post_hour"] = full["post_hour"].fillna(full["post_hour"].median())

full["post_date"] = pd.to_datetime(full["post_date"], errors="coerce")
full["day_of_week"] = full["post_date"].dt.dayofweek
full["is_weekend"] = full["day_of_week"].isin([5, 6]).astype(int)

full["day_of_week"] = full["day_of_week"].fillna(full["day_of_week"].median())
full["is_weekend"] = full["is_weekend"].fillna(0)

# FOLLOWERS + AD BOOST
full["followers"] = pd.to_numeric(full["followers"], errors="coerce").fillna(0)
full["followers_log"] = np.log1p(full["followers"])
full["ad_boost"] = full["ad_boost"].astype(int)

num_features = [
    "platform_id",
    "post_hour",
    "day_of_week",
    "is_weekend",
    "followers_log",
    "ad_boost"
]


Platform map: {0: 'Facebook', 1: 'Instagram', 2: 'TikTok'}


C:\Users\MSI\AppData\Local\Temp\ipykernel_17160\2548840900.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  full["post_hour"] = pd.to_datetime(full["post_time"], errors="coerce").dt.hour


In [5]:
for col in ["likes", "comments", "shares", "clicks"]:
    full[col] = np.log1p(full[col])

X_text = full["text"].values
X_num = full[num_features].values.astype("float32")

y_raw = full[targets].values.astype("float32")
y_scaler = StandardScaler()
y = y_scaler.fit_transform(y_raw)


In [6]:
X_text_tr, X_text_tmp, X_num_tr, X_num_tmp, y_tr, y_tmp = train_test_split(
    X_text, X_num, y, test_size=0.2, random_state=SEED
)

X_text_val, X_text_te, X_num_val, X_num_te, y_val, y_te = train_test_split(
    X_text_tmp, X_num_tmp, y_tmp, test_size=0.5, random_state=SEED
)


In [7]:
MAX_VOCAB = 30000
MAX_LEN = 80

tokenizer = Tokenizer(num_words=MAX_VOCAB, oov_token="<UNK>")
tokenizer.fit_on_texts(X_text_tr)

def seq_pad(x):
    return pad_sequences(
        tokenizer.texts_to_sequences(x),
        maxlen=MAX_LEN,
        padding="post",
        truncating="post"
    )

X_seq_tr = seq_pad(X_text_tr)
X_seq_val = seq_pad(X_text_val)
X_seq_te = seq_pad(X_text_te)


In [8]:
BATCH = 64

train_ds = tf.data.Dataset.from_tensor_slices(
    ((X_seq_tr, X_num_tr), y_tr)
).shuffle(len(X_seq_tr)).batch(BATCH)

val_ds = tf.data.Dataset.from_tensor_slices(
    ((X_seq_val, X_num_val), y_val)
).batch(BATCH)


In [9]:
EMB = 128
N_OUT = len(targets)

def inputs():
    return tf.keras.Input((MAX_LEN,)), tf.keras.Input((len(num_features),))

def embed(x):
    return tf.keras.layers.Embedding(MAX_VOCAB, EMB, mask_zero=True)(x)


In [10]:
def build_lstm():
    s, n = inputs()
    x = embed(s)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Concatenate()([x, n])
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    out = tf.keras.layers.Dense(N_OUT)(x)
    model = tf.keras.Model([s, n], out)
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss=tf.keras.losses.Huber())
    return model


In [11]:
def build_gru():
    s, n = inputs()
    x = embed(s)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Concatenate()([x, n])
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    out = tf.keras.layers.Dense(N_OUT)(x)
    model = tf.keras.Model([s, n], out)
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss=tf.keras.losses.Huber())
    return model


In [12]:
def build_transformer():
    s, n = inputs()
    x = embed(s)
    x = tf.keras.layers.MultiHeadAttention(4, EMB)(x, x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Concatenate()([x, n])
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    out = tf.keras.layers.Dense(N_OUT)(x)
    model = tf.keras.Model([s, n], out)
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
                  loss=tf.keras.losses.Huber())
    return model


In [13]:
def evaluate_and_print(model, name):
    y_pred = model.predict([X_seq_te, X_num_te])
    y_true = y_scaler.inverse_transform(y_te)
    y_pred = y_scaler.inverse_transform(y_pred)

    print(f"\n{name} PERFORMANCE")
    r2_list = []

    for i, t in enumerate(targets):
        mae = mean_absolute_error(y_true[:, i], y_pred[:, i])
        rmse = np.sqrt(mean_squared_error(y_true[:, i], y_pred[:, i]))
        r2 = r2_score(y_true[:, i], y_pred[:, i])
        r2_list.append(r2)

        print(f"{t:22s} | MAE={mae:.3f} | RMSE={rmse:.3f} | R2={r2:.4f}")

    print(f"➡ Average R² = {np.mean(r2_list):.4f}")


In [14]:
save_dir = r"D:\SLIIT Uni\Y4 S1\Research\Research Project\New train Model\backend\SavedModels"
os.makedirs(save_dir, exist_ok=True)


In [15]:
lstm_model = build_lstm()
lstm_model.fit(train_ds, validation_data=val_ds, epochs=36)
evaluate_and_print(lstm_model, "BiLSTM")
lstm_model.save(os.path.join(save_dir, "BiLSTM.keras"))
tf.keras.backend.clear_session()


Epoch 1/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 290s 183ms/step - loss: 0.1274 - val_loss: 0.0834
Epoch 2/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 307s 196ms/step - loss: 0.0880 - val_loss: 0.0918
Epoch 3/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 342s 219ms/step - loss: 0.0769 - val_loss: 0.0766
Epoch 4/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 321s 205ms/step - loss: 0.0713 - val_loss: 0.0746
Epoch 5/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 358s 229ms/step - loss: 0.0679 - val_loss: 0.0855
Epoch 6/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 360s 230ms/step - loss: 0.0661 - val_loss: 0.0726
Epoch 7/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 358s 229ms/step - loss: 0.0656 - val_loss: 0.0716
Epoch 8/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 361s 231ms/step - loss: 0.0646 - val_loss: 0.0727
Epoch 9/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 364s 233ms/step - loss: 0.0642 - val_loss: 0.0812
Epoch 10/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 366s 234ms/step - loss: 0.0640 - val_loss: 0.0730
Epoch 11/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 363s 232ms/step - loss: 0.0636 - val

In [16]:
gru_model = build_gru()
gru_model.fit(train_ds, validation_data=val_ds, epochs=36)
evaluate_and_print(gru_model, "BiGRU")
gru_model.save(os.path.join(save_dir, "BiGRU.keras"))
tf.keras.backend.clear_session()


Epoch 1/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 357s 226ms/step - loss: 0.1302 - val_loss: 0.0988
Epoch 2/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 320s 205ms/step - loss: 0.0914 - val_loss: 0.0911
Epoch 3/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 311s 199ms/step - loss: 0.0786 - val_loss: 0.0820
Epoch 4/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 317s 203ms/step - loss: 0.0723 - val_loss: 0.0870
Epoch 5/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 321s 205ms/step - loss: 0.0689 - val_loss: 0.0848
Epoch 6/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 336s 215ms/step - loss: 0.0672 - val_loss: 0.0754
Epoch 7/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 350s 224ms/step - loss: 0.0660 - val_loss: 0.0785
Epoch 8/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 337s 216ms/step - loss: 0.0655 - val_loss: 0.0758
Epoch 9/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 341s 218ms/step - loss: 0.0645 - val_loss: 0.0759
Epoch 10/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 352s 225ms/step - loss: 0.0639 - val_loss: 0.0729
Epoch 11/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 328s 210ms/step - loss: 0.0637 - val

In [17]:
trans_model = build_transformer()
trans_model.fit(train_ds, validation_data=val_ds, epochs=36)
evaluate_and_print(trans_model, "Transformer")
trans_model.save(os.path.join(save_dir, "Transformer.keras"))


Epoch 1/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 202s 128ms/step - loss: 0.2276 - val_loss: 0.0999
Epoch 2/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 214s 137ms/step - loss: 0.1171 - val_loss: 0.0912
Epoch 3/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 199s 127ms/step - loss: 0.1058 - val_loss: 0.0852
Epoch 4/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 197s 126ms/step - loss: 0.0981 - val_loss: 0.0806
Epoch 5/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 195s 125ms/step - loss: 0.0924 - val_loss: 0.0795
Epoch 6/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 185s 118ms/step - loss: 0.0880 - val_loss: 0.0784
Epoch 7/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 198s 126ms/step - loss: 0.0805 - val_loss: 0.0774
Epoch 8/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 195s 125ms/step - loss: 0.0754 - val_loss: 0.0761
Epoch 9/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 196s 125ms/step - loss: 0.0729 - val_loss: 0.0768
Epoch 10/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 198s 126ms/step - loss: 0.0713 - val_loss: 0.0771
Epoch 11/36
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 207s 132ms/step - loss: 0.0697 - val

In [18]:
with open(os.path.join(save_dir, "tokenizer.json"), "w") as f:
    f.write(tokenizer.to_json())

with open(os.path.join(save_dir, "y_scaler.pkl"), "wb") as f:
    pickle.dump(y_scaler, f)
